In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from wordcloud import WordCloud 
from sklearn.feature_extraction.text import TfidfVectorizer


stop_words = stopwords.words("english")

%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
listingsdf = pd.read_csv('../input/boston/listings.csv')
calendardf = pd.read_csv('../input/boston/calendar.csv')
reviewsdf  = pd.read_csv('../input/boston/reviews.csv')

In [3]:
# shape of the data sets
print('Listing - {0}, Calendar - {1}, Reviews - {2}'.format(listingsdf.shape, calendardf.shape, reviewsdf.shape))

In [4]:
listingsdf.head()

In [5]:
reviewsdf.head()

In [6]:
calendardf.head()

In [7]:
listingsdf.info()

In [8]:
reviewsdf.info()

In [9]:
calendardf.info()

## Listing data analysis

In [10]:
def listingdfValues(df=listingsdf, type='null', cutoff=0.01 ):
    if (type == 'null'):
        data = df.isna().sum().to_frame().reset_index()
    else:
        data = df.notna().sum().to_frame().reset_index()

    print('\n')
    data.rename(columns={0:'count'}, inplace=True)
    data['count_percentage'] = data['count'] / df.shape[0]

    data = data[data['count_percentage'] > cutoff]

    plt.figure(figsize=(38,12))
    plt.xticks(rotation=80)
    if (type == 'null') :      
        plt.title('Columns with percentage of null values in Listing dataset', fontsize=20 )
    else:
        plt.title('Columns with percentage of non-null values ', fontsize=20 )

    ax = sns.barplot(x='index', y='count_percentage', data=data )
    sns.despine()
    ax.set_xlabel('Columns in listing dataset', fontsize=20)
    ax.set_ylabel('percentage of Values', fontsize=20)

#     for container in ax.containers:
#         ax.bar_label(container*100, fmt='%.2f')

    for i in range (data.shape[0]):
        count = data.iloc[i]['count_percentage']

        # Refer here for details of the text() - https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.text.html
        plt.text(i, count + 0.02 , '{:0.1f}'.format(count*100), ha = 'center', va='center', rotation=90)

listingdfValues(listingsdf, 'null', 0.01)
listingdfValues(listingsdf, 'notnull', 0.0)

In [11]:
# data = listingsdf.notna().sum().to_frame().reset_index()
# data.rename(columns={0:'count'}, inplace=True)
# data['count_percentage'] = data['count'] / listingsdf.shape[0]

# data = data[data['count_percentage'] > 0.0]

# plt.figure(figsize=(30,12))
# plt.xticks(rotation=80)
# plt.title('Columns with percentage of non-null values ' )
# sns.barplot(x='index', y='count_percentage', data=data )
# plt.xlabel('Columns in listing dataset')
# plt.ylabel('percentage of Non-Null Values')


# for i in range (data.shape[0]):
#     count = data.iloc[i]['count_percentage']

#     # Refer here for details of the text() - https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.text.html
#     plt.text(i, count + 0.05 , '{:0.1f}'.format(count*100), rotation=90, ha = 'center', va='top')


In [12]:
# price column has $ sign and comma, create a new column by
listingsdf['price_cleansed'] = listingsdf.price.replace({'[\$,]':''}, regex=True).astype(float)

## Proportion of property types

In [13]:
def propertyTypes(df=listingsdf, byType='property_type'):
    
    if (byType == 'property_type'):
        rotation = 75
    else:
        rotation = 0
    
    data = listingsdf 
    data=listingsdf.groupby(byType)['id'].count().reset_index()
    total = data['id'].sum()
    data['percent'] = (data['id']/total * 100).round(2)

    plt.figure(figsize=(12,6))
    ax = sns.barplot(x=byType, y='id', data=data)
    sns.despine()
    
    byType = byType.replace("_"," ")
    plt.xticks(rotation=rotation)
    plt.title('count of the ' + byType, fontsize=24)
    ax.set_xlabel(byType, fontsize=16)
    ax.set_ylabel('count of ' + byType, fontsize=16)
    ax.grid(axis='y', linewidth=.4)

    for container in ax.containers:
        ax.bar_label(container, fontsize=14) #, fmt='%.2f')
        
    display(data.rename(columns={'id':'count'}).T)

propertyTypes(listingsdf, 'property_type')
propertyTypes(listingsdf, 'room_type')

In [14]:
# def propertyTypes(df=listingsdf):
    
#     data = listingsdf 
#     data=listingsdf.groupby('property_type')['id'].count().reset_index()
#     total = data['id'].sum()
#     data['percent'] = data['id']/total * 100

#     plt.figure(figsize=(12,6))
#     ax = sns.barplot(x='property_type', y='id', data=data)
#     sns.despine()
#     plt.xticks(rotation=75)
#     plt.title('count of the property types')
#     ax.set_xlabel('property types')
#     ax.set_ylabel('count of properties')

#     for container in ax.containers:
#         ax.bar_label(container) #, fmt='%.2f')
#     display(data.T)

# propertyTypes(listingsdf)

## Room Types

In [15]:

# data = listingsdf 
# data=listingsdf.groupby('room_type')['id'].count().reset_index()
# total = data['id'].sum()
# data['percent'] = data['id']/total * 100

# plt.figure(figsize=(8,4))
# ax = sns.barplot(y='room_type', x='id', data=data)
# sns.despine()
# # plt.xticks(rotation=75)
# plt.title('Type of rooms available')
# ax.set_xlabel('Count of rooms')
# ax.set_ylabel('Room Types')

# for container in ax.containers:
#     ax.bar_label(container) #, fmt='%.2f')


## type of rooms offered in different properties

In [16]:

data = listingsdf 
type_count = data[['property_type', 'room_type']].value_counts().reset_index()

# ax = plt.subplot()
plt.figure(figsize=(14,5))
ax = sns.barplot(x='property_type', y=0, hue='room_type', data=type_count)
sns.despine()
plt.xticks(rotation=75)
plt.title('property types vs room type count of properties', fontsize=20)
plt.legend(loc='upper right',fancybox=True)
ax.set_xlabel('properties type', fontsize=16)
ax.set_ylabel('properties count', fontsize=16)
ax.grid(axis='y', linewidth=.35)

for container in ax.containers:
    ax.bar_label(container) #, fmt='%.1f')

# for i in range(type_count.shape[0]):
#     count = type_count.iloc[i][0]
    
#     # Refer here for details of the text() - https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.text.html
# #     plt.text(i, count + 0.05 , '{:0.1f}'.format(count), rotation=90, ha = 'center', va='top')
    
#     plt.text(i, count + 0.05 , '{:0.1f}'.format(count), rotation=90, ha = 'center', va='top')



## Average price and range among the neighborhoods


In [17]:
def averagePriceNhood(df=listingsdf):
    '''
    accepts the dataframe, default is listingdf. 
    Calculate the mean price for each of the neighborhood
    
    PARAMETERS:
    df - dataframe, to use for calculating the average price of the neighbourhood
    '''

    pd.set_option('display.max_columns', 0)

    # data = listingsdf.groupby(['neighbourhood_cleansed','price_cleansed'])['city'].count().reset_index()
    data = listingsdf.sort_values('neighbourhood_cleansed') #.groupby(['neighbourhood_cleansed'])['price_cleansed'].mean().reset_index()

    # fig, ax = plt.subplots()
    plt.figure(figsize=(40,18))
    plt.xticks(rotation=90)

    ax = sns.countplot(x='neighbourhood_cleansed',  data=data)
    ax.set_ylabel('Number of properties', fontsize=20)
    ax.set_xlabel('neighborhoods',fontsize=20)
    for container in ax.containers:
        ax.bar_label(container, label_type = 'edge', fontsize=20) #, fmt='%.2f')
    ax.legend(loc=0)

    ax2=ax.twinx()
    ax = sns.lineplot(x='neighbourhood_cleansed', y='price_cleansed', legend=False, color='blue', ax=ax2, data=data)
    ax.set_ylabel('average price with variance', fontsize=20)
    ax.grid(axis='y', linewidth=0.4)
    ax.figure.legend()
    

    plt.title('count of properties (bars) and the average property price (line)',fontsize=20)

# plt.show()

    display(data.groupby('neighbourhood_cleansed')['price_cleansed'].mean().round(2)
            .to_frame().reset_index().rename(columns={'neighbourhood_cleansed':'neighborhood', 'price_cleansed':'average price'}).T)
    
averagePriceNhood(listingsdf)

In [18]:
listingsdf[listingsdf['neighbourhood_cleansed'] == 'Leather District'] #   ['price_cleansed'].sum() / 56

## price range in each neighborhood

In [19]:
def priceRange(df=listingsdf):
    plt.figure(figsize=(40,18))
    d = listingsdf[listingsdf['price_cleansed'] < 3000]
    ax = sns.boxplot(y='neighbourhood_cleansed', x='price_cleansed', data=d)
    ax.set_xlabel('price range', fontsize=20)
    ax.set_ylabel('neighborhoods', fontsize=20)
    plt.title('Neighborhood price ranges ', fontsize=20)
    ax.grid(axis='x', linewidth=0.45)


priceRange(listingsdf)
# ax = sns.swarmplot(y='neighbourhood', x='price_cleansed', data=d)

# Reviewers

## Reviewers staying at more than one property

In [20]:
def stayMoreThanOneProperty(df=reviewsdf, min_property_stay=3):
    '''
    find the reviewers that has stayed at more than single property.
    reviewsdf has reviewer-id and the property, where the reviewers had stayed and provided the feedback. Now, it is a case that an individual
    has stayed at the same property multiple times, and has left a review each time.
    Here we want to capture the unique properties that reviewer has stayed at.
    
    PARAMETERS:
    df : the reviewsdf dataframe, this will be a default, when no other df is passed
    min_property_stay: default is 3, and it will filter out only the reviewers that has stayed at least by this number of different properties
    
    RETURN:
    none
    '''

    min_property_stay = 3

    # group by reviewer and listing-id, to get unique reviewer-id, and listing-id pair
    data  = df.groupby(['reviewer_id','listing_id']).count().reset_index()

    # count the unique listing, reviewer stayed at
    data2 = data.groupby(['reviewer_id'])['listing_id'].count().to_frame().reset_index()

    # filter the result set
    data3 = data2[data2['listing_id'] >= min_property_stay]

    plt.figure(figsize=(9,4))
    plt.title('reviewers with atleast ' + str(min_property_stay) + ' different property stays' , fontsize=16)
    ax = sns.countplot(x='listing_id' , data=data3)
    ax.grid(axis='y', linewidth=.35)

    sns.despine()
    ax.set_xlabel('count of unique property stays', fontsize=14)
    ax.set_ylabel('number of reviewers', fontsize=14)
    for container in ax.containers:
        ax.bar_label(container, fontsize=12) #, fmt='%.2f')

stayMoreThanOneProperty(reviewsdf, 3)

## Reviews received by neighborhoods


In [21]:
reviews_neighborhood_df = reviewsdf.merge(listingsdf['neighbourhood_cleansed'], how='left', left_on=reviewsdf['listing_id'], right_on=listingsdf['id'] )

In [22]:
reviews_neighborhood_df.head()

In [23]:
def nhoodReviews(df=reviews_neighborhood_df):
    
    # data = reviews_neighborhood_df.groupby('neighbourhood_cleansed')['reviewer_id'].count()
    plt.figure(figsize=(16,10))
    # plt.xticks(rotation=90)
    plt.title('Neighborhood count of reviews', fontsize=20)
    ax = sns.countplot(y='neighbourhood_cleansed' , data=df)
    sns.despine()
    ax.set_xlabel('count of unique property stays', fontsize=20)
    ax.set_ylabel('number of reviewers', fontsize=20)
    ax.grid(axis='x', linewidth=.4)
    for container in ax.containers:
        ax.bar_label(container) #, fmt='%.2f')

nhoodReviews(reviews_neighborhood_df)

## most unique properties reviews

In [24]:
def mostUniquePropertiesReviews(df=reviewsdf, min_properties=7):
    
    min_reviews = 7
    # group by reviewer and listing-id, to get unique reviewer-id, and listing-id pair
    data  = df.groupby(['reviewer_id','listing_id']).count().reset_index()

    data2 = data.groupby(['reviewer_id'])['listing_id'].count().to_frame().reset_index()
    data2 = data2.merge(reviewsdf['reviewer_name'], how='left', left_on=data2['reviewer_id'], right_on=reviewsdf['reviewer_id'])
    data3 = data2[data2['listing_id'] >= min_reviews]
    plt.figure(figsize=(40,18))
    plt.xticks(rotation=90)
    plt.title('reviewers with atleast ' + str(min_reviews) + ' reviews', fontsize=22)
    ax = sns.barplot(x='reviewer_name', y='listing_id', data=data3)
    ax.set_xlabel('Reviewers', fontsize=20)
    ax.set_ylabel('Number of unique properties stay', fontsize=20)
    ax.grid(axis='y', linewidth='0.35')
    sns.despine()
    for container in ax.containers:
        ax.bar_label(container) #, fmt='%.2f')


mostUniquePropertiesReviews(reviewsdf, 7)

In [25]:
reviewsdf[reviewsdf['reviewer_id'] == 18607361].sort_values(['listing_id','date'])

In [26]:
reviews_neighborhood_df = reviewsdf.merge(listingsdf['neighbourhood_cleansed'], how='left', left_on=reviewsdf['listing_id'], right_on=listingsdf['id'] )
reviews_neighborhood_df.head()

In [28]:
# len(data3.reviewer_name.unique())

In [29]:
reviewsdf['reviewer_id'].value_counts()

In [30]:
neighborhoods = reviews_neighborhood_df['neighbourhood_cleansed'].unique()
neighborhoods

In [31]:
reviews_neighborhood_df[reviews_neighborhood_df['neighbourhood_cleansed'] == 'Downtown'].shape

In [40]:
# from tqdm import tqdm
# def review

vectorizer = TfidfVectorizer(ngram_range=(1,2))
lemmatizer = WordNetLemmatizer()
count = 0
for neighborhood in neighborhoods:
    print('\n\n')
    count += 1
    corpus = ''
    nhood = reviews_neighborhood_df[reviews_neighborhood_df['neighbourhood_cleansed'] == neighborhood]
    
#     for rec in tqdm(range(nhood.shape[0])):
    for rec in range(nhood.shape[0]):

    #     list_id  = reviewsdf.listing_id[rec]
        comments = reviewsdf.comments[rec]
        comments   = str(comments).replace('[\$,]',' ' )
        corpus += comments


    words = word_tokenize(corpus)
    tokens = [lemmatizer.lemmatize(word).lower().strip() for word in words if word.lower() not in stop_words]
#     tokens = [lemmatizer.lemmatize(word).lower().strip() for word in words ]
    
    corpus = " ".join(tokens)

    if (len(corpus) > 0):
        vectors = vectorizer.fit_transform([corpus])
        names = vectorizer.get_feature_names()

        data = vectors.todense().tolist()# Create a dataframe with the results

        df = pd.DataFrame(data, columns=names)

        wordcloud = WordCloud(background_color="white", max_words=100).generate_from_frequencies(df.T.sum(axis=1))
        plt.figure(figsize=(14,14))

        plt.imshow(wordcloud)
        plt.axis('off')
        plt.title('Neighborhood - ' + neighborhood, fontsize=24)
        plt.show()
        
    if (count > 4):
        break
        
print('done!')
# print(word_bag)

In [35]:
# stop_words